# Objectifs

- Lire les résultats des tests sur les 4 fichiers pour les corpus d'entraînement de D Longrée.
- Ajouter le nombre de tokens par corpus en lisant le fichier CSV lié ?

In [14]:
import glob
import os.path
import re
from collections import defaultdict

re_mname = re.compile(r"(?:model-longree-simpler-reprod-longree-simpler-|model-percent-reprod-longree-)([\w-]+)")
re_sub = re.compile(r"^([\w-]+):$")
re_metric = re.compile(r"^([\w-]+): ([\d\.]+)$")
def get_model_name(model_name):
    cnn_layer = "2"
    if "simpler" in model_name:
        cnn_layer = "1"
    grp = re_mname.findall(model_name)[0]
    return grp, cnn_layer 

#results = {
    #models : {
    #  {test-file: {metric: score}}
    #}
#}

results = defaultdict( # Key: Task
    lambda: defaultdict( # Key : model
        lambda: defaultdict( # Key : layers
            lambda: defaultdict( # Key : Test
                lambda: defaultdict( # Key Subtask
                    lambda: defaultdict( # Key Metric
                        str
                    )
                )
            )
        )
    )
)


header = [["model", "layers"]]
tests = []
metrics = []
models = []
tasks = defaultdict(list)

for file in glob.glob("/home/thibault/CloudStation/Thèse/Modeles/Resultats/Longree/*.txt"):
    basename = os.path.basename(file)
    model, test, _ = basename.split("_-_")
    model, date = model.split("---")
    model, layers = get_model_name(model)
    if (model, layers) not in models:
        models.append((model, layers))
    
    if test not in tests:
        tests.append(test)
        
    with open(file) as f:
        task, subtask = None, None
        for line in f:
            line = line.strip()
            if line.startswith("::: Evaluation report for task: "):
                task = line[len("::: Evaluation report for task: "): -4]
                continue
                
            if re_sub.match(line):
                subtask = re_sub.findall(line)[0]
                continue
                
            if re_metric.match(line):
                metric, number = re_metric.findall(line)[0]
                if (subtask, metric) not in tasks[task]:
                    tasks[task].append((subtask, metric))
                results[task][model][layers][test][subtask][metric] = number



In [15]:
import csv
for task, metrics in tasks.items():
    header = ["model", "layers", "test"] + ["/".join(m) for m in metrics]
    rows = [header]
    for test in sorted(tests):
        for model, layer in sorted(models, key=lambda x: x[0]+x[1]):
            row = [model, layer, test]
            for subtask, metric in metrics:
                row.append(results[task][model][layer][test][subtask][metric])
            rows.append(row)
    with open("longree-result-"+task+".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(rows)